In [ ]:
import numpy as np
from circuit_knitting.cutting.cut_finding.circuit_interface import SimpleGateList
from circuit_knitting.cutting.cut_finding.lo_cuts_optimizer import LOCutsOptimizer
from circuit_knitting.cutting.cut_finding.optimization_settings import OptimizationSettings
from circuit_knitting.cutting.cut_finding.quantum_device_constraints import DeviceConstraints

## Cut finding for efficient SU(2) Circuit with linear entanglement

### Visualize the circuit

In [ ]:
from qiskit.circuit.library import EfficientSU2
from circuit_knitting.cutting.cut_finding.utils import QCtoCCOCircuit

qc = EfficientSU2(4, entanglement="linear", reps=2).decompose()
qc.assign_parameters([0.4] * len(qc.parameters), inplace=True)

circuit_ckt=QCtoCCOCircuit(qc)

qc.draw("mpl", scale=0.8)




### Perform cut finding

In [ ]:
settings = OptimizationSettings(rand_seed = 12345)

settings.setEngineSelection('CutOptimization', 'BestFirst')


qubits_per_QPU=4
num_QPUs=2



for num_qpus in range(num_QPUs, 1, -1):
    for qpu_qubits in range(qubits_per_QPU, 1, -1):
        print(f'\n\n---------- {qpu_qubits} Qubits per QPU, {num_qpus} QPUs ----------')
        
        constraint_obj = DeviceConstraints(qubits_per_QPU = qpu_qubits, 
                                            num_QPUs       = num_QPUs)
        
        interface = SimpleGateList(circuit_ckt)

        op = LOCutsOptimizer(interface, 
                        settings, 
                        constraint_obj)
        
        out = op.optimize()

        print(' Gamma =', None if (out is None) else out.upperBoundGamma(),
              ', Min_gamma_reached =', op.minimumReached())
        if (out is not None):
            out.print(simple=True)
        else:
            print(out)
    
        print('Subcircuits:', interface.exportSubcircuitsAsString(name_mapping='default'),'\n')


## Cut finding for 7 qubit circuit

### Visualize the circuit

In [ ]:
from qiskit import QuantumCircuit
qc_0 = QuantumCircuit(7)
for i in range(7):
    qc_0.rx(np.pi / 4, i)
qc_0.cx(0, 3)
qc_0.cx(1, 3)
qc_0.cx(2, 3)
qc_0.cx(3, 4)
qc_0.cx(3, 5)
qc_0.cx(3, 6)

qc_0.draw("mpl")

### Perform cut finding

In [ ]:
from circuit_knitting.cutting.cut_finding.utils import QCtoCCOCircuit

circuit_ckt_wirecut=QCtoCCOCircuit(qc_0)

settings = OptimizationSettings(rand_seed = 12345)

settings.setEngineSelection('CutOptimization', 'BestFirst')

qubits_per_QPU=7
num_QPUs=2



for num_qpus in range(num_QPUs, 1, -1):
    for qpu_qubits in range(qubits_per_QPU, 1, -1):
        print(f'\n\n---------- {qpu_qubits} Qubits per QPU, {num_qpus} QPUs ----------')
        
        constraint_obj = DeviceConstraints(qubits_per_QPU = qpu_qubits, 
                                            num_QPUs       = num_QPUs)

        interface = SimpleGateList(circuit_ckt_wirecut)
        
        op = LOCutsOptimizer(interface, 
                        settings, 
                        constraint_obj)
        
        out = op.optimize()

        print(' Gamma =', None if (out is None) else out.upperBoundGamma(),
              ', Min_gamma_reached =', op.minimumReached())
        if (out is not None):
            out.print(simple=True)
        else:
            print(out)

        print('Subcircuits:', interface.exportSubcircuitsAsString(name_mapping='default'),'\n')